# Rheostat Reduced Model Attempt 14
In this notebook, we will try to get a good reduced model. Building off notes/comparisons/plots in attempt 13. <br>
7.22.2020

Notes to remember: <br>
- Want to keep the transient dynamics of ATP 
- Make the error metric dependent on ATP
- Timescale separation: remove things that quickly get to steady state, keep those that take longer
---

Run the sbml_to_ode function

In [1]:
%run sbml_to_ode2.py

Other imports

In [107]:
from auto_reduce import *
from auto_reduce.utils import get_ODE
import matplotlib.pyplot as plt
from auto_reduce.utils import reduce

import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()
from bokeh.layouts import row

import colorcet

Loading BokehJS ...

Last time we removed too many species.
## Plot Reduced Model 5: Remove all complexes except ATPase related (66,67)
___

In [58]:
filename = 'CRN.xml'
x,f,P,params_values,x_init = sbml_to_ode2(filename)

nouts = 1
n = len(x)
# Make ATP main output
C = np.zeros((nouts, n), dtype = int)
C[0][1] = 1 
C = C.tolist()

# Setup system
timepoints_ode = np.linspace(0,75,100)

error_tol = 100
nstates_tol = 1

sys = System(x, f, params = P, params_values = params_values, C = C, x_init = x_init)

sys_ode = get_ODE(sys, timepoints_ode)
sol = sys_ode.solve_system()

In [63]:
species_to_remove = [5,6,9,10,13, 14,17,18,23,24,28,29,31,32,
                 35,36,39,40,43,44,47,48,51,52,55,56,59,60,63,64]


In [64]:
timescale_list = []
for i in species_to_remove:
    timescale_list.append(x[i])

# Get appropriate list of complexes that are not in timescale_list
# What you should pass into solve_timescale_separation
update_timescale = []
for item in x:
    if item not in timescale_list:
        update_timescale.append(item)    
#update_timescale

In [65]:
timepoints_ssm = np.linspace(0,75,100)
timepoints_ode = np.linspace(0, 75, 100)
sys_reduce = reduce(sys, timepoints_ode, timepoints_ssm)

In [66]:
aa,bb = sys_reduce.solve_timescale_separation(update_timescale)
print('the ATP error is:',sys_reduce.get_error_metric(aa))

attempting to retain : [0, 1, 2, 3, 4, 7, 8, 11, 12, 15, 16, 19, 20, 21, 22, 25, 26, 27, 30, 33, 34, 37, 38, 41, 42, 45, 46, 49, 50, 53, 54, 57, 58, 61, 62, 65, 66, 67]
Successful time-scale separation solution obtained.
the ATP error is: 1377.5052417105946


In [67]:
sys_ode = get_ODE(aa, timepoints_ode)
sol = sys_ode.solve_system()

In [68]:
p = bokeh.plotting.figure(width = 475, height = 350, title = 'Reduced Model 5',
                             x_axis_label = 'Time (hrs)',
                             y_axis_label = 'Concentration (mM)')
#colors = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3']
colors = colorcet.b_glasbey_category10
p.line(timepoints_ode, sol[:,1],line_width = 2, legend_label = 'atp', color = colors[3])
p.line(timepoints_ode, sol[:,2], line_width = 2,legend_label = 'glucose', color = colors[0])
p.line(timepoints_ode, sol[:,34], line_width = 2,legend_label = 'isobutanol', color = colors[1])
p.legend.click_policy = 'hide'
bokeh.io.show(p)

Okay this is better than red mod 1. Let's take a look at all the species we have removed to see if there is anything else we should retain.

## Plot Full Model: Examine species that are retained in reduced model 5
Plot from **FULL MODEL SIMULATION**. Any species that reaches steady state quickly can be removed
___

In [87]:
# Get full model solution
filename = 'CRN.xml'
x,f,P,params_values,x_init = sbml_to_ode2(filename)

nouts = 1
n = len(x)
C = np.zeros((nouts, n), dtype = int)
C[0][1] = 1 
C = C.tolist()

# Setup system
timepoints_ode = np.linspace(0,75,1000)

#x_init = np.zeros(n)

error_tol = 100
nstates_tol = 1

sys = System(x, f, params = P, params_values = params_values, C = C, x_init = x_init)

sys_ode = get_ODE(sys, timepoints_ode)
sol = sys_ode.solve_system()

In [78]:
x_init[34]

0.0

Only plot the species that are removed in reduced model 5 from the full model simulation.

In [80]:
retained_species = [0, 1, 2, 3, 4, 7, 8, 11, 12, 15, 16, 19, 20, 21, 22, 25, 26, 27, 30, 33,
                    34, 37, 38, 41, 42, 45, 46, 49, 50, 53, 54, 57, 58, 61, 62, 65, 66, 67]

p4 = bokeh.plotting.figure(width = 375, height = 375, title = 'All Retained Species of Reduced Model 5',
                             x_axis_label = 'Time (hrs)',
                             y_axis_label = 'Concentration (mM)')
#colors = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3']
colors = colorcet.b_glasbey_category10
for i,ret_spec in enumerate(retained_species):
    p4.line(timepoints_ode, sol[:,ret_spec],line_width = 2, legend_label = str(ret_spec), color = colors[i])
    
p4.legend.click_policy = 'hide'
bokeh.io.show(p4)

Species that reaches ss before 30 (that aren't enzymes, have to keep enzymes, or other things w init conditions): 
- 3 g6p
- 8 f6p
- 12 f16p
- 16 g3p
- 21 3pg
- 27 13bpg
- 34 2pg
- 38 pep
- 42 pyk
- 46 acetolac
- 50 23dihmebut
- 54 3me2oxo
- 61 yahk
Take a look at a plot of these species to make sure they reach ss quickly

In [95]:
quick_ss = [3,8,12,16,21,27,34, 38, 42,46,50,54,61]

p5 = bokeh.plotting.figure(width = 375, height = 375, title = 'Quick ss species (that were retained in red mod 5)',
                             x_axis_label = 'Time (hrs)',
                             y_axis_label = 'Concentration (mM)')
#colors = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3']
colors = colorcet.b_glasbey_category10
for i,ret_spec in enumerate(quick_ss):
    p5.line(timepoints_ode, sol[:,ret_spec],line_width = 2, legend_label = str(ret_spec), color = colors[i])
    
p5.legend.click_policy = 'hide'
bokeh.io.show(p5)

## Plot Full Model: Examine species that were removed in reduced model 5
___

In [10]:
filename = 'CRN.xml'
x,f,P,params_values,x_init = sbml_to_ode2(filename)

nouts = 1
n = len(x)
C = np.zeros((nouts, n), dtype = int)
C[0][1] = 1 
C = C.tolist()

# Setup system
timepoints_ode = np.linspace(0,75,1000)

#x_init = np.zeros(n)

error_tol = 100
nstates_tol = 1

sys = System(x, f, params = P, params_values = params_values, C = C, x_init = x_init)

sys_ode = get_ODE(sys, timepoints_ode)
sol = sys_ode.solve_system()

p1 = bokeh.plotting.figure(width = 475, height = 350, title = 'Species removed in reduced model 5',
                             x_axis_label = 'Time (hrs)',
                             y_axis_label = 'Concentration (mM)')
p2 = bokeh.plotting.figure(width = 475, height = 350, title = 'Full Model from SBML',
                             x_axis_label = 'Time (hrs)',
                             y_axis_label = 'Concentration (mM)')

#colors = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3']
colors = colorcet.b_glasbey_category10
p2.line(timepoints_ode, sol[:,1],line_width = 2, legend_label = 'atp', color = colors[3])
p2.line(timepoints_ode, sol[:,2], line_width = 2,legend_label = 'glucose', color = colors[0])
p2.line(timepoints_ode, sol[:,62], line_width = 2,legend_label = 'isobutanol', color = colors[1])

species_to_remove = [5,6,9,10,13, 14,17,18,23,24,28,29,31,32,
                 35,36,39,40,43,44,47,48,51,52,55,56,59,60,63,64]
for i,spec in enumerate(species_to_remove):
    p1.line(timepoints_ode, sol[:,spec], line_width = 2, color = colors[i], 
            legend_label = str(species_to_remove[i]))
# p_reg.line(timepoints_ode, sol[:,43], line_width = 2,legend_label = 'random1', color = colors[4])
# p_reg.line(timepoints_ode, sol[:,14], line_width = 2,legend_label = 'random2', color = colors[5])
# p_reg.line(timepoints_ode, sol[:,5], line_width = 2,legend_label = 'random3', color = colors[6])

#p_reg.legend.click_policy = 'hide'
bokeh.io.show(row(p1,p2))

Maybe we should retain purple one? **28** -     complex_enzyme_mGapDH_2x_metabolite_nadp_metabolite_pi_2x_molecule_g3p

## Plot Reduced Model 6: Remove all complexes, retain 66,67,28
---

In [11]:
filename = 'CRN.xml'
x,f,P,params_values,x_init = sbml_to_ode2(filename)

nouts = 1
n = len(x)
# Make ATP main output
C = np.zeros((nouts, n), dtype = int)
C[0][1] = 1 
C = C.tolist()

# Setup system
timepoints_ode = np.linspace(0,75,100)

error_tol = 100
nstates_tol = 1

sys = System(x, f, params = P, params_values = params_values, C = C, x_init = x_init)

sys_ode = get_ODE(sys, timepoints_ode)
sol = sys_ode.solve_system()

In [38]:
species_to_remove = [5,6,9,10,13, 14,17,18,23,24,31,32,29,
                 35,36,39,40,43,44,47,48,51,52,55,56,59,60,63,64]

timescale_list = []
for i in species_to_remove:
    timescale_list.append(x[i])

# Get appropriate list of complexes that are not in timescale_list
# What you should pass into solve_timescale_separation
update_timescale = []
for item in x:
    if item not in timescale_list:
        update_timescale.append(item)    
#update_timescale

In [39]:
timepoints_ssm = np.linspace(0,75,100)
timepoints_ode = np.linspace(0, 75, 100)
sys_reduce = reduce(sys, timepoints_ode, timepoints_ssm)

aa,bb = sys_reduce.solve_timescale_separation(update_timescale)
print('the ATP error is:',sys_reduce.get_error_metric(aa))

attempting to retain : [0, 1, 2, 3, 4, 7, 8, 11, 12, 15, 16, 19, 20, 21, 22, 25, 26, 27, 28, 30, 33, 34, 37, 38, 41, 42, 45, 46, 49, 50, 53, 54, 57, 58, 61, 62, 65, 66, 67]
Successful time-scale separation solution obtained.
the ATP error is: inf


In [40]:
sys_ode = get_ODE(aa, timepoints_ode)
sol = sys_ode.solve_system()

In [45]:
p3 = bokeh.plotting.figure(width = 475, height = 350, title = 'Reduced Model 6',
                             x_axis_label = 'Time (hrs)',
                             y_axis_label = 'Concentration (mM)')
#colors = ['#66c2a5','#fc8d62','#8da0cb','#e78ac3']
colors = colorcet.b_glasbey_category10
p3.line(timepoints_ode, sol[:,1],line_width = 2, legend_label = 'atp', color = colors[3])
p3.line(timepoints_ode, sol[:,2], line_width = 2,legend_label = 'glucose', color = colors[0])
p3.line(timepoints_ode, sol[:,33], line_width = 2,legend_label = 'isobutanol', color = colors[1])
p3.legend.click_policy = 'hide'
bokeh.io.show(p3)

Uh huh weird, if we try to retain species 28, things go bonkers. Even if we also try to retain species 29.

## Plot Reduced Model 7: Will remove all complexes (except 66,67) and remove other species that reach ss before time 30
Note that species with initial conditions are left in even if they reach ss before 30
___

In [99]:
filename = 'CRN.xml'
x,f,P,params_values,x_init = sbml_to_ode2(filename)

nouts = 1
n = len(x)
# Make ATP main output
C = np.zeros((nouts, n), dtype = int)
C[0][1] = 1 
C = C.tolist()

# Setup system
timepoints_ode = np.linspace(0,75,100)

error_tol = 100
nstates_tol = 1

sys = System(x, f, params = P, params_values = params_values, C = C, x_init = x_init)

sys_ode = get_ODE(sys, timepoints_ode)
sol = sys_ode.solve_system()

In [105]:
# species_to_remove = [5,6,9,10,13, 14,17,18,23,24,28,29,31,32,
#                  35,36,39,40,43,44,47,48,51,52,55,56,59,60,63,64,
#                     3,8,12,16,21,27,34, 38, 42,46,50,54,61]

species_to_remove = [5,6,9,10,13, 14,17,18,23,24,28,29,31,32,
                 35,36,39,40,43,44,47,48,51,52,55,56,59,60,63,64,
                    54]


timescale_list = []
for i in species_to_remove:
    timescale_list.append(x[i])

# Get appropriate list of complexes that are not in timescale_list
# What you should pass into solve_timescale_separation
update_timescale = []
for item in x:
    if item not in timescale_list:
        update_timescale.append(item)    
#update_timescale

In [106]:
timepoints_ssm = np.linspace(0,75,100)
timepoints_ode = np.linspace(0, 75, 100)
sys_reduce = reduce(sys, timepoints_ode, timepoints_ssm)

aa,bb = sys_reduce.solve_timescale_separation(update_timescale)
print('the ATP error is:',sys_reduce.get_error_metric(aa))

attempting to retain : [0, 1, 2, 3, 4, 7, 8, 11, 12, 15, 16, 19, 20, 21, 22, 25, 26, 27, 30, 33, 34, 37, 38, 41, 42, 45, 46, 49, 50, 53, 57, 58, 61, 62, 65, 66, 67]
The time-scale separation that retains states [0, 1, 2, 3, 4, 7, 8, 11, 12, 15, 16, 19, 20, 21, 22, 25, 26, 27, 30, 33, 34, 37, 38, 41, 42, 45, 46, 49, 50, 53, 57, 58, 61, 62, 65, 66, 67] does not work because the state-variables [complex_enzyme_IlvD_molecule_3me2oxo] appear in the reduced model


AttributeError: 'NoneType' object has no attribute 'x'

In [104]:
type(aa)

NoneType

In [ ]:
sys_ode = get_ODE(aa, timepoints_ode)
sol = sys_ode.solve_system()

RUN INTO PROBLEMS HERE WITH ALGORITHM SOLVING

## Conclusion
___

We will stick with reduced model 5: the one where all complexes are removed except 66, 67, the ones involved in ATP use. The reason for this is because those species take longer to reach steady state so it was sacrificing the dynamics of ATP too much. <br>
After investigating this reduced model, we saw that complex 28 takes longer to reach steady state than expected and it was removed. However, when we tried another reduced model with this removed, the results were unexplainable. *May have to investigate this again.* <br>
We also saw that there are a number of intermediate species that could be removed because they are retained in red mod 5 but reach ss quickly. However, when we tried to simulate this, we ran into problems with the algorithm. This is most likely due to the fact that too many species are being removed. We can keep these in for now. It makes more intuitive sense to remove the complexes and keep the intermediate species to me since those are actual species whose initial conditions could be more directly manipulated.